# Automating a processing chain using Python apps (scripts)
So far, we have run Jupyter Notebooks to go through the steps involved in the creation of a baseline image composite, the downloading and processing of change detection images, the classification into land cover maps, the change detection and verification, and the aggregation of many change maps into one report image file. We have then vectorised the report image to reduce its file size and emailed the reports to registered users as an email attachment.
This notebook shows how we can build an efficient and more automated forest alerts processing chain. We will do this by basically turning the four previous notebooks into Python scripts (also sometimes called Apps). A Python script ends with .py and is executable from within Python.

# Setup: Requirements to use this Notebook

Check which directory we are in.

In [ ]:
pwd

Go to the pyeo home directory.

In [1]:
import os
pyeo_dir = '/home/cmsstudent/pyeo'
os.chdir(pyeo_dir)
workdir = os.getcwd()
print(workdir)
config_path = os.path.join(pyeo_dir, 'pyeo_linux_azure.ini')
print(config_path)

/home/cmsstudent/pyeo
/home/cmsstudent/pyeo/pyeo_linux_azure.ini


We did this in the previous notebook step-by-step. Here, we initialie the notebook in one code cell to speed up the process.

In [2]:
import argparse
import configparser
import cProfile
import datetime
from datetime import datetime
import email
from email import encoders
from email.mime.base import MIMEBase
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.message import EmailMessage
import geopandas as gpd
import glob
import json
import numpy as np
import pandas as pd
import os
from osgeo import gdal
import shutil
import smtplib
import ssl
import sys
import warnings
import zipfile

from pyeo import (
    classification, 
    filesystem_utilities,
    queries_and_downloads, 
    raster_manipulation,
    vectorisation
)
from pyeo.filesystem_utilities import config_to_log
from pyeo.acd_national import (
    acd_initialisation,
    acd_config_to_log,
    acd_roi_tile_intersection
)

gdal.UseExceptions()
warnings.simplefilter("ignore", category=UserWarning)

print("Libraries successfully imported")

/home/cmsstudent/miniconda3/envs/pyeo_env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/cmsstudent/miniconda3/envs/pyeo_env/lib/python3.10/abc.py:106: SHDeprecationWarning: AWS functionality will remain in the codebase for now, but won't be actively maintained.
  cls = super().__new__(mcls, name, bases, namespace, **kwargs)


Libraries successfully imported


# make_composite.py

This App downloads and preprocesses Sentinel 2 images into a median image composite.

Usage from a terminal command line (we add the % to call it from a Jupyter Notebook):

>%run pyeo/apps/change_detection/make_composite.py /home/cmsstudent/pyeo/pyeo_linux_azure.ini

In this configuration, all processing parameters will be taken from the .ini file.

>%run pyeo/apps/change_detection/make_composite.py /home/cmsstudent/pyeo/pyeo_linux_azure.ini --tile 36NXG

If the above --tile flag is specified, the App will override the selection of tiles from the geojson location file with the specified Sentinel-2 tile ID.

In [3]:
%run pyeo/apps/change_detection/make_composite.py /home/cmsstudent/pyeo/pyeo_linux_azure.ini

2024-10-25 16:16:22,593: INFO: ---------------------------------------------------------------
2024-10-25 16:16:22,595: INFO: ---                 PROCESSING START                        ---
2024-10-25 16:16:22,595: INFO: ---------------------------------------------------------------
2024-10-25 16:16:22,596: INFO: conda environment path found: /home/cmsstudent/miniconda3//envs/pyeo_env
2024-10-25 16:16:22,597: INFO: True
2024-10-25 16:16:22,600: INFO: Config file that controls the processing run: /home/cmsstudent/pyeo/pyeo_linux_azure.ini
2024-10-25 16:16:22,601: INFO: ---------------------------------------------------------------
2024-10-25 16:16:22,602: INFO:   run_mode :  watch_period_seconds
2024-10-25 16:16:22,603: INFO:   forest_sentinel :  model
2024-10-25 16:16:22,604: INFO:   environment :  sen2cor_path
2024-10-25 16:16:22,604: INFO:   raster_processing_parameters :  change_to_classes
2024-10-25 16:16:22,605: INFO:   vector_processing_parameters :  minimum_area_to_report_m2
2

/home/cmsstudent/Desktop/pyeo_data/tilelist.csv already exists. Renaming the old file.
New file name: /home/cmsstudent/Desktop/pyeo_data/tilelist_2.csv


2024-10-25 16:16:24,074: INFO:     57 L1C products
2024-10-25 16:16:24,075: INFO:     43 L2A products
2024-10-25 16:16:24,075: INFO: --------------------------------------------------
2024-10-25 16:16:24,076: INFO: Filtering out L2A products that have the same 'start_date'  as another L2A product with a different processing baseline.  Dropping baseline N9999 where it is duplicating an existing baseline.
2024-10-25 16:16:24,136: INFO: Removing 3 L2A products with duplicate acquisition dates.
2024-10-25 16:16:24,188: INFO:   43 L2A products remain after removing duplicate acquisition dates:
2024-10-25 16:16:24,190: INFO:        S2A_MSIL2A_20220116T075251_N9999_R135_T36NXG_20221201T131716.SAFE
2024-10-25 16:16:24,191: INFO:        S2A_MSIL2A_20220106T075321_N0301_R135_T36NXG_20220106T112039.SAFE
2024-10-25 16:16:24,192: INFO:        S2A_MSIL2A_20220129T080201_N0400_R035_T36NXG_20220129T105531.SAFE
2024-10-25 16:16:24,193: INFO:        S2A_MSIL2A_20220126T075211_N0400_R135_T36NXG_20220126T

runtime analysis saved to /home/cmsstudent/Desktop/pyeo_data/make_composite.prof
Run snakeviz over the profile file to interact with the profile information:
>snakeviz /home/cmsstudent/Desktop/pyeo_data/make_composite.prof


# detect_change.py
This App downloads and preprocesses Sentinel 2 images for change detection and classifies them using a machine learning model. It performs change detection against a baseline median image composite. Generates a report image file and optionally vectorises it if selected in the ini file.

Usage from a terminal command line (we add the % to call it from a Jupyter Notebook):

>%run pyeo/apps/change_detection/detect_change.py /home/cmsstudent/pyeo/pyeo_linux_azure.ini

In this configuration, all processing parameters will be taken from the .ini file.

>%run pyeo/apps/change_detection/detect_change.py /home/cmsstudent/pyeo/pyeo_linux_azure.ini --tile 36NXG

If the --tile flag is specified, the App will override the selection of tiles from the geojson location file with the specified Sentinel-2 tile ID.

In [5]:
# navigate to the pyeo root directory
os.chdir(pyeo_dir)

In [ ]:
%run pyeo/apps/change_detection/detect_change.py /home/cmsstudent/pyeo/pyeo_linux_azure.ini

2024-10-25 17:33:55,908: INFO: ---------------------------------------------------------------
2024-10-25 17:33:55,909: INFO: ---                 PROCESSING START                        ---
2024-10-25 17:33:55,910: INFO: ---------------------------------------------------------------
2024-10-25 17:33:55,910: INFO: conda environment path found: /home/cmsstudent/miniconda3//envs/pyeo_env
2024-10-25 17:33:55,911: INFO: ---------------------------------------------------------------
2024-10-25 17:33:55,912: INFO: Config file: /home/cmsstudent/pyeo/pyeo_linux_azure.ini
2024-10-25 17:33:55,912: INFO: ---------------------------------------------------------------
2024-10-25 17:33:55,913: INFO: ----------------------------
2024-10-25 17:33:55,913: INFO: Contents of the config file:
2024-10-25 17:33:55,914: INFO: ----------------------------
2024-10-25 17:33:55,915: INFO:   run_mode :  watch_period_seconds
2024-10-25 17:33:55,915: INFO:   forest_sentinel :  model
2024-10-25 17:33:55,916: INFO:

/home/cmsstudent/Desktop/pyeo_data/tilelist.csv already exists. Renaming the old file.
New file name: /home/cmsstudent/Desktop/pyeo_data/tilelist_3.csv


2024-10-25 17:33:57,297: INFO: Removed 0 faulty scenes <0MB in size from the list
2024-10-25 17:33:57,302: INFO:     60 L1C products
2024-10-25 17:33:57,303: INFO:     40 L2A products
2024-10-25 17:33:57,304: INFO: Capping the number of L1C products to 3
2024-10-25 17:33:57,305: INFO: Relative orbits found covering tile: [ 35 135]
2024-10-25 17:33:57,309: INFO:     2 L1C products remain:
2024-10-25 17:33:57,310: INFO:        S2A_MSIL1C_20240316T074651_N0510_R135_T36NXG_20240316T093624.SAFE
2024-10-25 17:33:57,312: INFO:        S2B_MSIL1C_20240314T075709_N0510_R035_T36NXG_20240314T094856.SAFE
2024-10-25 17:33:57,312: INFO: Number of L1C products for dataspace is 2
2024-10-25 17:33:57,314: INFO: Capping the number of L2A products to 3
2024-10-25 17:33:57,315: INFO: Relative orbits found covering tile: [ 35 135]
2024-10-25 17:33:57,319: INFO:     2 L2A products remain:
2024-10-25 17:33:57,320: INFO:        S2A_MSIL2A_20240316T074651_N0510_R135_T36NXG_20240316T110248.SAFE
2024-10-25 17:33:

# send_report.py
This App sends out any new vectorised report files (.shp format) to a list of recipients specified in an email list file. Currently only Gmail is implemented. WhatsApp will be added in the future. Options are set in the .ini file and login details in the credentials file. Any newly created shapefiles found will be zipped and emailed out as an attachment. The zipping process ensures that they are not sent out twice.

Usage from a terminal command line (we add the % to call it from a Jupyter Notebook):

>%run pyeo/apps/change_detection/send_report.py /home/cmsstudent/pyeo/pyeo_linux_azure.ini

In this configuration, all processing parameters will be taken from the .ini file.

>%run pyeo/apps/change_detection/send_report.py /home/cmsstudent/pyeo/pyeo_linux_azure.ini --tile 36NXG

If the --tile flag is specified, the App will override the selection of tiles from the geojson location file with the specified Sentinel-2 tile ID.

In [ ]:
# navigate to the pyeo root directory
os.chdir(pyeo_dir)

In [17]:
%run pyeo/apps/change_detection/send_report.py /home/cmsstudent/pyeo/pyeo_linux_azure.ini

2024-08-15 21:19:07,783: INFO: ---------------------------------------------------------------
2024-08-15 21:19:07,783: INFO: ---                 PROCESSING START                        ---
2024-08-15 21:19:07,783: INFO: ---------------------------------------------------------------
2024-08-15 21:19:07,795: INFO: conda environment path found: C:\Users\hb91\.conda\envs\pyeo_env
2024-08-15 21:19:07,808: INFO: Config file that controls the processing run: Z:\gy7709\pyeo\pyeo_windows_gy7709.ini
2024-08-15 21:19:07,814: INFO: ---------------------------------------------------------------
2024-08-15 21:19:07,819: INFO: ----------------------------
2024-08-15 21:19:07,823: INFO: Contents of the config file:
2024-08-15 21:19:07,828: INFO: ----------------------------
2024-08-15 21:19:07,833: INFO:   run_mode :  watch_period_seconds
2024-08-15 21:19:07,839: INFO:   forest_sentinel :  model
2024-08-15 21:19:07,843: INFO:   environment :  sen2cor_path
2024-08-15 21:19:07,848: INFO:   raster_pro

Z:\gy7709\tilelist.csv already exists. Renaming the old file.
New file name: Z:\gy7709\tilelist_12.csv


2024-08-15 21:19:09,935: INFO: Region of interest processing based on ROI file.
2024-08-15 21:19:09,946: INFO: Saved Sentinel-2 tile ID list: Z:\gy7709\tilelist_12.csv
2024-08-15 21:19:09,948: INFO: 2 Sentinel-2 tiles to process.
2024-08-15 21:19:09,953: INFO: Sending out the latest reports for Sentinel-2 tile: 36NXG
2024-08-15 21:19:09,959: INFO: Z:\gy7709\36NXG
2024-08-15 21:19:11,990: INFO: Redirecting log output to tile log: Z:\gy7709\36NXG\log\36NXG.log
2024-08-15 21:19:12,134: INFO: ---------------------------------------------------------------
2024-08-15 21:19:12,146: INFO: ---                 PROCESSING START                        ---
2024-08-15 21:19:12,147: INFO: ---------------------------------------------------------------
2024-08-15 21:19:12,149: INFO: ---------------------------------------------------------------
2024-08-15 21:19:12,154: INFO: ---  TILE PROCESSING START: 36NXG                          ---
2024-08-15 21:19:12,157: INFO: --------------------------------

SystemExit: 1